In [1]:
import os, subprocess, shutil
import numpy as np

import parmed as pmd

from pmx import forcefield as pmxff

# Choose underlying toolkits for the OpenFF toolkit
Choose the toolkit(s) you want to use with the OpenFF toolkit (either OpenEye or Ambertools and RDKit)

In [2]:
from openforcefield.utils import toolkits

### OpenEye version: uncomment the following if you have and if you want to use the OpenEye toolkit, then RDKit and Ambertools toolkits
# toolkit_precedence = [toolkits.OpenEyeToolkitWrapper, toolkits.RDKitToolkitWrapper, toolkits.AmberToolsToolkitWrapper]

### Non-OpenEye version: uncomment the following if you want to use the rdkit and ambertools
toolkit_precedence = [toolkits.RDKitToolkitWrapper, toolkits.AmberToolsToolkitWrapper]

toolkits.GLOBAL_TOOLKIT_REGISTRY = toolkits.ToolkitRegistry(toolkit_precedence=toolkit_precedence)

In [3]:
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import ForceField

In [4]:
# choose the open force field version:
# examples: 'smirnoff99Frosst-1.1.0.offxml' 'openff-1.0.0.offxml'
forcefield = 'openff-1.0.0.offxml'
openff_forcefield = ForceField(forcefield)

In [5]:
# function to convert openFF molecule (ligand) to a parmed structure
def ligandToPMD(ligand):
    ligand_positions = ligand.conformers[0]
    
    # Calculate am1bcc charges and fix them such that they add up to zero (4 digit precision)
    try:
        ligand.compute_partial_charges_am1bcc()
    except Exception as e:
        raise Exception('Error in charge calculation for ligand {}: {}'.format(ligand.name, e))
    # Give all atoms unique names so we can export to GROMACS
    for idx, atom in enumerate(ligand.atoms):
        atom.name = f'{atom.element.symbol}{idx}'
    
    # Do not assign H-bond constraints now, instead have ParmEd add them later
    del openff_forcefield._parameter_handlers['Constraints']

    ligand_topology = ligand.to_topology()
    try:
        ligand_system = openff_forcefield.create_openmm_system(ligand_topology, charge_from_molecules=[ligand])
    except Exception as e:
        raise Exception('Error in creating openmm system: {}'.format(e))
    # Create OpenMM Topology from OpenFF Topology
    omm_top = ligand_topology.to_openmm()


    # Convert OpenMM System to a ParmEd structure.
    pmd_structure = pmd.openmm.load_topology(omm_top, ligand_system, ligand_positions)

    return pmd_structure, ligand_topology, ligand_system, ligand_positions

In [6]:
# functions for Gromacs force field  manuplation/conversion
def set_charge_to_zero(itp):
    q = 0.0
    n = 0
    for a in itp.atoms:
        q += a.q
        n += 1
    intq = round(q)
    diffq = intq - q
    # round to 6 digit precision
    deltaq = np.around(diffq/float(n), decimals=6)
    
    newq = 0.0
    for a in itp.atoms:
        a.q += deltaq
        a.q = np.around(a.q, decimals=6)
        newq += a.q
    # add remainder to first atom
    intq = round(newq)
    diffq = intq - newq
    itp.atoms[0].q += diffq

def change_atomtypes(itp,suffix):
    for a in itp.atoms:
#        newtype = str(a.atomtype)+'_'+str(a.name)+str(suffix)
        newtype = str(a.atomtype)+str(suffix)
        a.atomtype = newtype

    newdict = {}
    for atkey in itp.atomtypes.keys():
        at = itp.atomtypes[atkey]
        newtype = str(atkey)+str(suffix)
        newdict[newtype] = at
    itp.atomtypes=newdict

def write_ff(atypes, fname, ff='amber99sb'):
    fp = open(fname,'w')
    print('[ atomtypes ]', file=fp)
    for atkey in atypes.keys():
        at = atypes[atkey]
        print('%8s %12.6f %12.6f %3s %12.6f %12.6f' % (atkey, at[1], at[2], at[3], at[4], at[5]), file=fp)

def write_posre(itp, fname, fc=1000):
    fp = open(fname,'w')
    print('[ position_restraints ]', file=fp)
    for i, atom in enumerate(itp.atoms):
        print("%d   1    %d   %d    %d" % (i+1,fc,fc,fc), file=fp)
    fp.close()


# Convert SDF files to gromacs topologies

If you don't have SDF files, you first need to convert them (i.e. with the OpenEye toolkit OEChem)

ATTENTION: Using PDB files might prone to errors because the pdb files do not have bond information

In [7]:
for target_id, target in enumerate(['jnk1', 'pde2', 'thrombin']):
    print('=== ' + target + ' ===')
    with open(f'./input/{target_id+1:02d}_{target}/ligands.txt') as f:
        ligNames = f.read().splitlines()
    for ligName in ligNames:
        ligPath= f'systems/{target_id+1:02d}_{target}/03_docked/{ligName}/'
        topPath= f'systems/{target_id+1:02d}_{target}/04_topo/{forcefield}/{ligName}/'  
        os.makedirs(f'{topPath}', exist_ok=True)
        
        if os.path.isfile(f'{ligPath}/{ligName}.sdf'):
            ligand = Molecule.from_file(f'{ligPath}/{ligName}.sdf')
        elif os.path.isfile(f'{ligPath}/{ligName}.pdb'):
            # Try to read in PDB file instead of a SDF, only works with OpenEye
            ligand = Molecule.from_file(f'{ligPath}/{ligName}.pdb')
            # save as sdf file 
            # ATTENTION: automatic conversion to SDF
            ligand.to_file(f'{ligPath}/{ligName}.sdf', 'sdf')
        else:
            print('    File not found. Molecules {} cannot be read in'.format(ligName))
            continue
        
        print('   ', ligName)  
        try: 
            pmd_structure, ligand_topology, ligand_system, ligand_positions = ligandToPMD(ligand)
        except Exception as e:
            print('    ' + str(e))
            continue
            
        # Export GROMACS files.
        pmd_structure.save(f'{topPath}/{ligName}.top', overwrite = True)
        pmd_structure.save(f'{topPath}/{ligName}.gro', overwrite = True, precision = 8)

        # Export AMBER files.
        pmd_structure.save(f'{topPath}/{ligName}.prmtop', overwrite=True)
        pmd_structure.save(f'{topPath}/{ligName}.inpcrd', overwrite=True)
        
        # Create GROMACS ITP file
        itp = pmxff.read_gaff_top(f'{topPath}/{ligName}.top')
        itp.set_name('MOL')
        change_atomtypes(itp, ligName)
        set_charge_to_zero(itp)

        itpout = ligName + '.itp'
        posre = 'posre_' + ligName + '.itp'
        itp.write(f'{topPath}/{itpout}')
        write_ff(itp.atomtypes, f'{topPath}/ff{itpout}')
        write_posre(itp, f'{topPath}/{posre}') 

=== jnk1 ===
    lig_17124-1
    lig_18624-1
    lig_18625-1
    lig_18626-1
    lig_18627-1
    lig_18628-1
    lig_18629-1
    lig_18630-1
    lig_18631-1
    lig_18632-1
    lig_18633-1
    lig_18634-1
    lig_18635-1
    lig_18636-1
    lig_18637-1
    lig_18638-1
    lig_18639-1
    lig_18652-1
    lig_18658-1
    lig_18659-1
    lig_18660-1
=== pde2 ===
    lig_43249674
    lig_48009208
    lig_48022468
    lig_48168913
    lig_48271249
    lig_49072088
    lig_49137374
    lig_49137530
    lig_49175789
    lig_49175828
    lig_49220392
    lig_49220548
    lig_49396360
    lig_49580115
    lig_49582390
    lig_49582468
    lig_49585367
    lig_49932129
    lig_49932714
    lig_50107616
    lig_50181001
=== thrombin ===
    lig_1a
    lig_1b
    lig_1c
    lig_1d
    lig_3a
    lig_3b
    lig_5
    lig_6a
    lig_6b
    lig_6e
    lig_7a


# Generate GAFF2 topologies

In [10]:
forcefield = 'gaff2'
for target_id, target in enumerate(['jnk1', 'pde2', 'thrombin']):
    print('=== ' + target + ' ===')
    with open(f'./input/{target_id+1:02d}_{target}/ligands.txt') as f:
        ligNames = f.read().splitlines()
    for ligName in ligNames:
        print('   ', ligName)  
        ligPath= f'systems/{target_id+1:02d}_{target}/03_docked/{ligName}/'
        topPath= f'systems/{target_id+1:02d}_{target}/04_topo/{forcefield}/{ligName}/'  
        os.makedirs(f'{topPath}', exist_ok=True)
        
        if os.path.isfile(f'{ligPath}/{ligName}.pdb'):
            os.makedirs(f'{topPath}', exist_ok=True)
            # quick hack to get formal charge of molecule
            itp = pmxff.read_gaff_top(f'systems/{target_id+1:02d}_{target}/04_topo/smirnoff99Frosst-1.1.0.offxml/{ligName}/{ligName}.top')
            intq = round(sum([a.q for a in itp.atoms]))
            print(f'Formal Charge: {int(intq)} e')
            
            subprocessOutput = subprocess.run(f'python ./scriptpath/acpype.py \
                                                -i {ligPath}/{ligName}.pdb \
                                                -o gmx \
                                                -a gaff2 \
                                                -n {int(intq)}'.split(),
                                                capture_output = True)
            print(subprocessOutput.stdout.decode('utf8') + subprocessOutput.stderr.decode('utf8'))
            shutil.copy(f'{ligName}.acpype/{ligName}_GMX.itp', topPath)
            shutil.copy(f'{ligName}.acpype/{ligName}_GMX.gro', topPath)
            shutil.copy(f'{ligName}.acpype/{ligName}_GMX.top', topPath) 
            
            if os.path.isdir(f'{topPath}/{ligName}.acpype'):
                shutil.rmtree(f'{topPath}/{ligName}.acpype')
            shutil.move(f'{ligName}.acpype', f'{topPath}')
            
            itp = pmxff.read_gaff_top(f'{topPath}/{ligName}_GMX.itp')
            itp.set_name('MOL')

            itpout = ligName + '.itp'
            posre = 'posre_' + ligName + '.itp'
            
            itp.write(f'{topPath}/{itpout}')
            write_ff(itp.atomtypes, f'{topPath}/ff{itpout}')
            write_posre(itp, f'{topPath}/{posre}')

=== jnk1 ===
Formal Charge: 0 e
| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'LIG'....
Checking parameters for unit 'LIG'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
++++++++++end_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
==> * Tleap OK *
==> Removing temporary files...
==> Writing GROMACS files

==> Writing GMX dihedrals for GMX 4.5 and higher.

==> Writing pickle file lig_17124-1.pkl
Total time of execution: 2m 18s

Formal Charge: 0 e
| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamb

| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'LIG'....
Checking parameters for unit 'LIG'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
++++++++++end_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
==> * Tleap OK *
==> Removing temporary files...
==> Writing GROMACS files

==> Writing GMX dihedrals for GMX 4.5 and higher.

==> Writing pickle file lig_18632-1.pkl
Total time of execution: 1m 14s

Formal Charge: 0 e
| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==>

| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'LIG'....
Checking parameters for unit 'LIG'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
++++++++++end_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
==> * Tleap OK *
==> Removing temporary files...
==> Writing GROMACS files

==> Writing GMX dihedrals for GMX 4.5 and higher.

==> Writing pickle file lig_18658-1.pkl
Total time of execution: 2m 49s

Formal Charge: 0 e
| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==>

| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'Q2T'....
Checking parameters for unit 'Q2T'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
++++++++++end_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
==> * Tleap OK *
==> Removing temporary files...
==> Writing GROMACS files

==> Writing GMX dihedrals for GMX 4.5 and higher.

==> Writing pickle file lig_49137374.pkl
Total time of execution: 2m 2s

Formal Charge: 0 e
| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==>

| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'Q2T'....
Checking parameters for unit 'Q2T'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
++++++++++end_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
==> * Tleap OK *
==> Removing temporary files...
==> Writing GROMACS files

==> Writing GMX dihedrals for GMX 4.5 and higher.

==> Writing pickle file lig_49582468.pkl
Total time of execution: 2m 50s

Formal Charge: 0 e
| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==

| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set to 1
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'LIG'....

/home/dhahn3/miniconda3/envs/off/bin/teLeap: Warning!
The unperturbed charge of the unit (1.001000) is not zero.
Checking parameters for unit 'LIG'.
Checking for bond parameters.
Checking for angle parameters.
check:  Warnings: 1
Unit is OK.
++++++++++end_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
==> * Tleap OK *
==> Removing temporary files...
==> Writing GROMACS files

==> Writing GMX dihedrals for GMX 4.5 and higher.

==> Writing pickle file lig_1d.pkl
Total time of execution: 3m 39s

Formal Charge: 1 e
| ACPYPE: AnteChamber PYthon Parser interfacE v. 0 0 Rev: 0 (c) 2019 AWSdS |
==> ... charge set t